In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
from fastai.vision.all import *

labels = pd.read_csv("../input/dog-breed-identification/labels.csv")
labels


In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_ids, valid_ids = next(split.split(labels, labels["breed"]))
labels["is_valid"] = [i in valid_ids for i in range(len(labels))]

labels["id"] = labels["id"].apply(lambda x: x + ".jpg")


In [ ]:
path = "../input/dog-breed-identification/train"

dls = ImageDataLoaders.from_df(labels, path,
                               item_tfms=Resize(460, method="squeeze"),
                               batch_tfms=[*aug_transforms(size=300),
                                           Normalize.from_stats(*imagenet_stats)],
                               bs=32, valid_col="is_valid")
dls.show_batch()


In [ ]:
xb, yb = dls.one_batch()


In [ ]:
densenet = nn.Sequential(*list(densenet169(pretrained=True).children())[:-1], 
                         nn.AdaptiveAvgPool2d((1, 1)),
                         nn.Flatten())


In [ ]:
resnet = nn.Sequential(*list(resnet152(pretrained=True).children())[:-1], 
                       nn.Flatten())


In [ ]:
class NeuralNet(Module):
    def __init__(self, extractors, hidden_size, vocab_size, device):
        
        self.extractors = extractors
        for conv in self.extractors:
            conv.eval().to(device)
            
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.5),
            nn.Linear(512, vocab_size)).to(device)
        
    def forward(self, x):
        
        features = torch.cat([conv(x) for conv in self.extractors], dim=1)
        
        return self.classifier(features)


In [ ]:
extractors = [densenet, resnet]
hidden_size = 1664 + 2048
device = "cuda" if torch.cuda.is_available() else "cpu"
model = NeuralNet(extractors, hidden_size, len(dls.vocab), device)


In [ ]:
learn = Learner(dls, model, metrics=accuracy, path=".").to_fp16()
learn.lr_find()


In [ ]:
learn.fit_one_cycle(3, 1e-3)


In [ ]:
torch.cuda.empty_cache()


In [ ]:
test_files = get_image_files("../input/dog-breed-identification/test")
test_dl = dls.test_dl(test_files, bs=16)


In [ ]:
preds, targs = learn.tta(dl=test_dl)


In [ ]:
#preds = torch.softmax(preds, dim=1)
sub = pd.DataFrame({"id":test_files.map(lambda x:x.stem)})
sub[list(dls.vocab)] = preds
sub.to_csv("submission.csv", index=False)
